# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Loading the csv exported in Part I to a DataFrame

In [2]:
file = "/Users/enriquevazquez/Documents/BootCamp TEC/Homework/06-Python-APIs/Instructions/output_data/cities1.csv"
city_data_df = pd.read_csv(file)
city_data_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Kapaa,40.0,US,1.585857e+09,69.0,22.08,-159.32,299.15,2.60
1,San Carlos,86.0,VE,1.585857e+09,32.0,9.66,-68.58,308.50,1.68
2,Castro,99.0,BR,1.585857e+09,52.0,-24.79,-50.01,295.84,1.69
3,Jamestown,1.0,US,1.585857e+09,45.0,42.10,-79.24,284.15,5.10
4,Saint-Philippe,75.0,RE,1.585857e+09,94.0,-21.36,55.77,296.15,3.60


### Humidity Heatmap
* Configuring gmaps.
* Using the Lat and Lng as locations and Humidity as the weight.
* Adding Heatmap layer to map.

In [3]:
locations = city_data_df[['Lat','Lng']]
humidity = city_data_df['Humidity']
fig = gmaps.figure()
h_l = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=150, point_radius=3)
fig.add_layer(h_l)
fig

Figure(layout=FigureLayout(height='420px'))

### Creating new DataFrame fitting weather criteria
* Narrowing down the cities to fit weather conditions.
* Droping any rows will null values.

In [4]:
ideal_temp = city_data_df.loc[(city_data_df['Max Temp'] < 290) & (city_data_df['Max Temp'] > 280) & 
                              (city_data_df['Cloudiness'] > 40) & (city_data_df['Cloudiness'] < 80) & 
                              (city_data_df['Wind Speed'] > 3) & (city_data_df['Wind Speed'] < 9)]
ideal_temp['Hotel Name'] = ''
ideal_temp

/Users/enriquevazquez/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
110,Villablino,55.0,ES,1.585858e+09,70.0,42.94,-6.32,282.59,3.10,
115,Pueblo,75.0,US,1.585858e+09,43.0,38.25,-104.61,285.37,4.60,
134,Heiligenhafen,75.0,DE,1.585858e+09,70.0,54.37,10.98,281.15,6.20,
139,Sechelt,75.0,CA,1.585858e+09,64.0,49.47,-123.77,281.48,4.10,
153,Ñuñoa,51.0,PE,1.585858e+09,18.0,-14.48,-70.64,285.79,4.19,
338,Coshocton,75.0,US,1.585858e+09,49.0,40.27,-81.86,288.15,3.60,
365,Ermelo,41.0,ZA,1.585858e+09,57.0,-26.53,29.98,281.34,3.35,
415,İmamoğlu,75.0,TR,1.585859e+09,100.0,37.27,35.66,289.82,3.10,
446,Selimpaşa,75.0,TR,1.585859e+09,87.0,41.06,28.38,280.15,6.20,
450,Arrondissement de Lannion,75.0,FR,1.585859e+09,87.0,48.75,-3.42,282.15,3.10,


### Hotel Map
* Storing into variable named `hotel_df`.
* Adding a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Storing the first Hotel result into the DataFrame.
* Ploting markers on top of the heatmap.

In [5]:
for index,row in ideal_temp.iterrows():
    target_search = 'Hotel'
    lat = ideal_temp.loc[index,['Lat']]
    lng = ideal_temp.loc[index,['Lng']]
    params = {
        'location':f'{lat[0]},{lng[0]}',
        'types': target_search,
        'radius':'5000',
        'key': g_key
    }
    url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    hotels = requests.get(url,params).json()
    ideal_temp.loc[index,['Hotel Name']] = hotels['results'][1]['name']

hotel_df = ideal_temp[['Hotel Name','Lat','Lng']]

for index,row in hotel_df.iterrows():
    target_search = 'Hotel'
    lat = hotel_df.loc[index,['Lat']]
    lng = hotel_df.loc[index,['Lng']]
    params = {
        'location':f'{lat[0]},{lng[0]}',
        'types': target_search,
        'radius':'5000',
        'key': g_key
    }
    url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    hotels = requests.get(url,params).json()
    hotel_df.loc[index,['Lat']] = hotels['results'][1]['geometry']['location']['lat']
    hotel_df.loc[index,['Lng']] = hotels['results'][1]['geometry']['location']['lng']

/Users/enriquevazquez/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [6]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in ideal_temp.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [7]:
# Adding marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
#fig = gmaps.figure()
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))